In [23]:
from tensorflow.keras.preprocessing import image
def process_img_path(img_path):
    return image.load_img(img_path, target_size=(224, 224))

In [22]:
data = []
for i in ['phoenix', 'boston', 'saopaolo', 'tokyo', 'trondheim']:
    for file in os.listdir('../input/smalldatanew/smallData/train/'+i):
#         print(file) #to check if right files...
        if file[-3:] == 'jpg':
            path = os.path.join(f'./data/{i}/' + file)
            img = process_img_path(path)
            x = image.img_to_array(img)
            
            data.append(x)
#             print(x)

In [ ]:
#imports
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
#instantiate resnet50 & sew own a head -- input layer shape 
res = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
# sew my tail on this beast
x1 = res.output
x1 = GlobalAveragePooling2D()(x1)
predictions = Dense(y_train[0].shape[0], activation='softmax')(x1)
model = Model(res.input, predictions)

In [26]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.densenet import decode_predictions
from keras.applications.densenet import preprocess_input

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import keras.preprocessing.image

import numpy as np

# Basic Convolution

In [27]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [28]:
img_width, img_height = 640, 480

train_data_dir = '../input/smalldatanew/smallData/train'
validation_data_dir = '../input/smalldatanew/smallData/validation'
nb_train_samples = 1000
nb_validation_samples = 250
epochs = 10
batch_size = 16

In [29]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

# Transfer Learning

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [30]:
data_dir = '../input/smalldatanew/smallData/train'
img_height, img_width=224,224
batch_size=32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [31]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [12]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(6):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

In [32]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224, 224, 3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

In [33]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(5, activation='softmax'))

In [19]:
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=Adam(lr=0.001), loss="sparse_categorical_crossentropy"
, metrics=['accuracy'])

history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=10)

In [36]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [49]:
# DOESN'T WORK

import cv2
img_path = '../input/tokyotest/tokyotokyo.jpg'
test = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))

pred=resnet_model.predict(image)

output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)

In [ ]:
try:
    path=os.path.join(mypath, n)
    img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img, (img_rows, img_cols))

except Exception as e:
    print(str(e))